In [ ]:
from ocpy.ocp import OCP
from ocpy.ddp import DDPSolver
from ocpy import symutils

import numpy as np
import sympy as sym
from sympy import sin, cos, tan, exp, log, ln, sinh, cosh, tanh, diff, sqrt


In [ ]:
n_x = 12
n_u = 6
sim_name = 'hexacopter'

ocp = OCP(n_x, n_u, sim_name)

t = ocp.get_t()
x = ocp.get_x()
u = ocp.get_u()


In [ ]:
m, l, k, Ixx, Iyy, Izz, gamma, g = ocp.define_scalar_constants(
    [('m', 1.44), ('l', 0.23), ('k', 1.6e-09),
     ('Ixx', 0.0348), ('Iyy', 0.0459), ('Izz', 0.0977),
     ('gamma', 0.01), ('g', 9.80665)]
)
z_ref, u_min, u_max, epsilon = ocp.define_scalar_constants(
    [('z_ref', 5), ('u_min', 0.144), ('u_max', 6), ('epsilon', 0.01)]
)

q = ocp.define_vector_constant(
    'q',  [1, 1, 1, 0.01, 0.01, 0, 0.01, 0.01, 0.01, 0.1, 0.1, 0.001])
r = ocp.define_vector_constant(
    'r',  [0.01, 0.01, 0.01, 0.01, 0.01, 0.01])
qf = ocp.define_vector_constant(
    'qf',  [1, 1, 1, 0.01, 0.01, 0, 0.01, 0.01, 0.01, 0.1, 0.1, 0.001])
Q = sym.diag(*q)
R = sym.diag(*r)
Qf = sym.diag(*qf)


In [ ]:
p_ref = ocp.zero_vector(3)
p_ref[0] = sin(2*t)
p_ref[1] = cos(2*t)
p_ref[2] = z_ref + 2*sin(2*t)
# Or, directly
# p_ref = sym.Matrix([[sin(2*t)],
#                     [(1 - cos(2*t))],
#                     [z_ref + 2*sin(t)]])
p_ref_diff = p_ref.diff(t)

x_ref = ocp.zero_vector(n_x)
x_ref[0:3, :] = p_ref
x_ref[3:6, :] = p_ref_diff

U1 = sum(u[i] for i in range(n_u))
U2 = l*(-u[0]/2 - u[1] - u[2]/2 + u[3]/2 + u[4]+ u[5]/2)
U3 = l*(-(sqrt(3)/2)*u[0] + (sqrt(3)/2)*u[2] + (sqrt(3)/2)*u[3] - (sqrt(3)/2)*u[5])
U4 = k*(-u[0] + u[1] - u[2] + u[3] - u[4] + u[5]) - gamma * x[11]

f = ocp.zero_vector(n_x)
f[0] = x[6]
f[1] = x[7]
f[2] = x[8]
f[3] = x[9]
f[4] = x[10]
f[5] = x[11]
f[6] = (cos(x[5])*sin(x[4])*cos(x[3]) + sin(x[5])*sin(x[3]))*U1/m
f[7] = (sin(x[5])*sin(x[4])*cos(x[3]) - cos(x[5])*sin(x[3]))*U1/m
f[8] = -g + (cos(x[3])*cos(x[4]))*U1/m
f[9] = ((Iyy-Izz)/Ixx)*x[10]*x[11] + U2/Ixx
f[10] = ((Izz-Ixx)/Iyy)*x[9]*x[11] + U3/Iyy
f[11] = ((Ixx-Iyy)/Izz)*x[9]*x[10] + U4/Izz

u_ref = ocp.zero_vector(n_u)
for i in range(n_u):
    u_ref[i] = (m*g) / 6

u_barrier = sum(-ln(u[i] - u_min) - ln(u_max - u[i]) for i in range(n_u))
l = (x - x_ref).T * Q * (x - x_ref) + (u - u_ref).T * R * (u - u_ref) + epsilon * sym.Matrix([u_barrier])
lf = (x - x_ref).T * Qf * (x - x_ref)

display('f', f)
display('l', l)
display('lf', lf)

In [ ]:
T = 5.0
N = 200

t0 = 0.0
x0 = np.array([0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0])
us_guess = np.array([[1, 1, 1, 1, 1, 1] for i in range(N)])

ocp.define(f, l, lf, T, N, t0, x0, us_guess)


In [ ]:
solver = DDPSolver(ocp)


In [ ]:
alphas = [0.5**i for i in range(8)].append(0)
solver.set_solver_parameters(
    max_iter=300, alphas=alphas, damp_init=1e-1, damp_max=1e4, damp_min=1e-3,
    stop_threshold=1e-2
)
# solve ocpj
ts, xs, us, Js = solver.solve(result= True, log=True)

In [ ]:
tval = 0.0
xval = np.random.random(len(x))

df, dl = ocp.get_derivatives()
lfxx_numba = dl[-1]
lfxx_numba(xval, tval)